In [86]:
%pwd

'd:\\'

In [87]:
import os
os.chdir("../")

In [88]:
import os
print(os.listdir('d:\\doctor_gpt_final'))

['.devcontainer', '.env', '.git', '.github', '.gitignore', 'app.py', 'Data', 'Generative_AI_Project.egg-info', 'LICENSE', 'pytest.ini', 'README.md', 'requirements.txt', 'research', 'setup.py', 'src', 'template.py', 'test.py', 'tests']


In [89]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [90]:
import os
import pandas as pd
from langchain.docstore.document import Document

def load_csv_file(data, input_column='input', output_column='output'):
    """Load CSV files from a directory and return LangChain Documents."""
    documents = []
    if not os.path.exists(data):
        raise FileNotFoundError(f"Directory '{data}' does not exist. Please check the path.")
    csv_files = [f for f in os.listdir(data) if f.endswith('.csv')]
    if not csv_files:
        raise ValueError(f"No CSV files found in directory '{data}'.")
    for csv_file in csv_files:
        file_path = os.path.normpath(os.path.join(data, csv_file))
        print(f"Processing CSV file: {file_path}")
        try:
            # Use encoding_errors instead of errors
            df = pd.read_csv(file_path, encoding='utf-8', encoding_errors='replace')
            required_columns = [col for col in [input_column, output_column] if col]
            missing_cols = [col for col in required_columns if col not in df.columns]
            if missing_cols:
                raise ValueError(f"Missing columns {missing_cols} in {file_path}. Available: {df.columns.tolist()}")
            for index, row in df.iterrows():
                input_text = row.get(input_column, '') if input_column in df.columns else ''
                output_text = row.get(output_column, '') if output_column in df.columns else ''
                # Combine input and output, skip if both are empty or null
                text = ' '.join([t for t in [str(input_text), str(output_text)] if t and not pd.isna(t)])
                if not text:
                    print(f"Warning: Skipping row {index} in {file_path} due to empty or null content.")
                    continue
                metadata = {
                    'source': file_path,
                    'row_index': index,
                    '__index__': row.get('__index_level_0__', '') if '__index_level_0__' in df.columns else '',
                    'instruction': row.get('instruction', '') if 'instruction' in df.columns else ''
                }
                documents.append(Document(page_content=text, metadata=metadata))
        except pd.errors.ParserError:
            print(f"Error: Failed to parse CSV file '{file_path}'. Skipping.")
            continue
    return documents

In [91]:
import pandas as pd
df = pd.read_csv('d:\\doctor_gpt_final\\Data\\train.csv')
print("Columns:", df.columns.tolist())
print("First few rows:\n", df.head())
print("Null values:\n", df.isnull().sum())

Columns: ['instruction', 'input', 'output', '__index_level_0__']
First few rows:
                                          instruction  \
0  If you are a doctor, please answer the medical...   
1  If you are a doctor, please answer the medical...   
2                    Answer this question truthfully   
3  If you are a doctor, please answer the medical...   
4  If you are a doctor, please answer the medical...   

                                               input  \
0  Hey Just wondering.  I am a 39 year old female...   
1  I had a brain anyurism in 1993. I used to have...   
2  When to seek urgent medical care when I have U...   
3  Hi My problem is that before some month my ski...   
4  Im having horrible pain in my lower abdominal,...   

                                              output  __index_level_0__  
0  hello and thank you for using chatbot. i caref...             157271  
1  hello brain tumor may present in various ways....             235698  
2  Call your health ca

In [92]:
extracted_data = load_csv_file(data='d:\\doctor_gpt_final\\Data\\', input_column='input', output_column='output')

Processing CSV file: d:\doctor_gpt_final\Data\train.csv


In [94]:
#extracted_data

In [95]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [96]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 382881


In [97]:
from langchain.embeddings import HuggingFaceEmbeddings

In [98]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [99]:
embeddings = download_hugging_face_embeddings()

In [100]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [101]:
#query_result

In [102]:
from dotenv import load_dotenv
import os
success = load_dotenv(dotenv_path='d:\\doctor_gpt_final\\.env')  # Adjust path
print("Loaded .env:", success)
if not success:
    raise FileNotFoundError("Failed to load .env file. Check the file path and contents.")
print("PINECONE_API_KEY:", os.getenv('PINECONE_API_KEY')[:5] + "...")
print("GROQ_API_KEY:", os.getenv('GROQ_API_KEY')[:5] + "...")

Loaded .env: True
PINECONE_API_KEY: pcsk_...
GROQ_API_KEY: gsk_U...


In [103]:
success = load_dotenv(dotenv_path='d:\\gita\\.env')

In [104]:
from dotenv import load_dotenv
load_dotenv()

False

In [105]:
from pinecone import Pinecone, ServerlessSpec
import os

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "doctorgpt"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [106]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [42]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [43]:
docsearch

In [44]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
retrieved_docs = search_db(user_query="What are the symptoms of diabetes?")

In [46]:
retrieved_docs

[Document(id='37165b4c-a078-4ac1-8cc7-fdc96fd06dbd', metadata={'page': 1.0, 'source': 'data\\Bhagavad-gita_1.pdf'}, page_content='His cosmic form, It has a properly complicated view of the ways of the\nreligious life and treats of the paths of knowledge, works, discipline\nand faith and their inter-relationships, problems that have bothered\nadherents of other religions in other times and places. The devotion\nspoken of is a deliberate means of religious satisfaction, not a mere\noutpouring of poetic emotion. Next to the Bhagavata-purana, a long work\nfrom South India, the Gita is the text most frequently quoted in the'),
 Document(id='bb370e3d-48fc-405b-98dd-05025f0c203b', metadata={'page': 418.0, 'source': 'data\\Bhagavad-gita_1.pdf'}, page_content='processes, speculation, rituals, sacrifices, charities, etc. That is\nthe specific benediction of devotional service.\nSimply by chanting the holy name of Krsna--Hare Krsna, Hare Krsna,\nKrsna Krsna, Hare Hare/ Hare Rama, Hare Rama, Rama 

In [47]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from loguru import logger

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [48]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "doctorgpt"
index = pc.Index(index_name)

In [50]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [51]:
def search_db(user_query: str) -> list:
    sim_docs =[]
    result = vector_store.similarity_search_with_score(
    user_query, k=3
    )
    for doc in result:
        sim_docs.append(doc[0].page_content)
    
    return sim_docs    

In [ ]:
retrieved_docs = search_db(user_query="What are the causes of hypertension?")

In [53]:
search

["of life in spite of all difficulties. The difficulties usually arise\nfrom having to sever family relationships, to give up the connection of\nwife and children. But if anyone is able to tolerate such difficulties,\nsurely his path to spiritual realization is complete. Similarly, in\nArjuna's discharge of duties as a ksatriya, he is advised to persevere,\neven if it is difficult to fight with his family members or similarly",
 'Here Lord Krsna tells Arjuna that he can do as he chooses. God does not\ninterfere with the little independence of the living entity. In\nBhagavad-gita, the Lord has explained in all respects how one can\nelevate his living condition. The best advice imparted to Arjuna is to\nsurrender unto the Supersoul seated within his heart. By right\ndiscrimination, one should agree to act according to the order of the\nSupersoul. That will help one become situated constantly in Krsna',
 'kingdom on earth with sovereignty like the demigods in heaven.\nPURPORT\nAlthough Ar

In [54]:
# ---------------------------- LLM --------------------------------------
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.5)

In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from loguru import logger

# Load environment variables from .env file
load_dotenv()

# Validate GROQ_API_KEY
groq_api_key = os.getenv("GROQ_API_KEY")
if groq_api_key is None:
    logger.error("GROQ_API_KEY environment variable is not set.")
    raise ValueError("GROQ_API_KEY environment variable is not set. Please set it in your environment or Streamlit Cloud secrets.")

# Set GROQ_API_KEY in os.environ (required by langchain_groq)
os.environ['GROQ_API_KEY'] = groq_api_key

# ---------------------------- LLM --------------------------------------
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)

def doctor_gpt_ai(user_query: str, doc_list: list) -> str:
    template = """
    You are a health consultant AI chatbot, Doctor GPT. Your role is to provide accurate and reliable answers to user questions about health conditions, symptoms, treatments, or any health-related topics typically asked of a doctor, based on the provided documents. Use the information from the `doc_list` to address the `user_query` thoroughly and correctly. Ensure that your response is:

    - **Accurate:** Base your answers solely on the information in the provided documents.
    - **Conversational:** Maintain a friendly and approachable tone.
    - **Mature and Consultancy-Oriented:** Present information in a professional and trustworthy manner.

    **Inputs:**
    1. `user_query`: {user_query} The question posed by the user about a health condition or related topic.
    2. `doc_list`: {doc_list} A list of documents containing relevant information related to the user's question.

    **Instructions:**
    - Analyze the `user_query` and identify the key information needed to answer it.
    - Review the `doc_list` to find relevant information that addresses the query.
    - Construct a response that is clear, concise, and directly answers the user's question using the information from the documents.
    - Avoid introducing information not present in the `doc_list`.
    - If the `user_query` has no relevant information in the `doc_list`, return "I'm sorry, I don't have information on that health topic. Please ask a question related to health conditions or topics covered in my documents."
    - If the `user_query` is an empty string, respond with "Please provide a valid health condition query."
    - Maintain a tone that is both professional and empathetic, suitable for a consultancy setting.

    Return the answer as the only output.
    Always make sure that you're returning the answer without any explanation.
    The output should be the answer alone.
    Always return this: "Please provide a valid health condition query." for empty query.
    """
    question_prompt = PromptTemplate(input_variables=["user_query", "doc_list"], template=template)
    initiator_router = question_prompt | llm | StrOutputParser()
    output = initiator_router.invoke({"user_query": user_query, "doc_list": doc_list})
    return output

In [ ]:
answer_1 = doctor_gpt_ai(user_query="What are the symptoms of asthma?", doc_list=search_db(user_query="What are the symptoms of asthma?"))

In [ ]:
def doctor_gpt_ai(user_query: str) -> str:
    logger.info("Searching for relevant documents in the medical vector database")
    doc_list = search_db(user_query=user_query)
    logger.info("Calling the doctor_gpt_ai to generate a medical response")
    answer = doctor_gpt_ai(user_query=user_query, doc_list=doc_list)
    logger.info("Returning the final medical answer")
    return answer


In [ ]:
question_1 = "What are the most common causes of chronic diseases?"
question_2 = "How does diabetes affect the body over time?"
question_3 = "What are the early warning signs of cancer?"
question_4 = "How do infectious diseases spread between individuals?"
question_5 = "What lifestyle changes can help prevent heart disease?"


In [ ]:
answer_1 = doctor_gpt_ai(user_query=question_1)
print(answer_1)

2025-05-23 07:04:12.802 | INFO     | __main__:gita_chatbot:2 - Searching for similar docs in the Bhagavad Gita vector DB
2025-05-23 07:04:14.564 | INFO     | __main__:gita_chatbot:4 - Calling the gitabot_ai to get answer
2025-05-23 07:04:15.113 | INFO     | __main__:gita_chatbot:6 - Returning the final answer


The central message of the Bhagavad Gita is to deliver mankind from the nescience of material existence, helping individuals to understand and overcome the anxieties and difficulties that arise from it, as seen in the example of Arjuna's surrender to Sri Krsna, which led to the revelation of the Bhagavad Gita.


In [ ]:
answer_2 = doctor_gpt_ai(user_query=question_2)
print(answer_2)

2025-05-23 07:04:26.098 | INFO     | __main__:gita_chatbot:2 - Searching for similar docs in the Bhagavad Gita vector DB
2025-05-23 07:04:26.512 | INFO     | __main__:gita_chatbot:4 - Calling the gitabot_ai to get answer
2025-05-23 07:04:27.465 | INFO     | __main__:gita_chatbot:6 - Returning the final answer


Krishna teaches that karma is not eternal and that it can be changed by adopting the right sort of activities. According to Krishna, if one performs their prescribed duties according to their nature, they will not be affected by sinful reactions. Additionally, Krishna advises Arjuna to act in Krsna consciousness, giving up the results of fruitive activities, which will lead to the bondage of birth being broken and ultimately achieving a state of liberation. This means that by performing one's duties selflessly and with devotion, one can transcend the cycle of karma and achieve spiritual freedom.


In [ ]:
answer_3 = doctor_gpt_ai(user_query=question_3)
print(answer_3)

2025-05-23 07:04:39.986 | INFO     | __main__:gita_chatbot:2 - Searching for similar docs in the Bhagavad Gita vector DB
2025-05-23 07:04:40.341 | INFO     | __main__:gita_chatbot:4 - Calling the gitabot_ai to get answer
2025-05-23 07:04:41.050 | INFO     | __main__:gita_chatbot:6 - Returning the final answer


The concept of dharma in the Bhagavad Gita is understood as the principles of religion and morality. It is mentioned that the instructions of Bhagavad-gita constitute the supreme process of religion and of morality, with the last instruction being the last word in all morality and religion: surrender unto Krsna. Dharma is also related to the paths of knowledge, works, discipline, and faith, and their inter-relationships, which are discussed in the Gita as means to achieve religious satisfaction and lead people to God consciousness and obedience to the principles of religion.


In [ ]:
answer_4 = doctor_gpt_ai(user_query=question_4)
print(answer_4)

2025-05-23 07:05:11.015 | INFO     | __main__:gita_chatbot:2 - Searching for similar docs in the Bhagavad Gita vector DB
2025-05-23 07:05:11.401 | INFO     | __main__:gita_chatbot:4 - Calling the gitabot_ai to get answer
2025-05-23 07:05:12.135 | INFO     | __main__:gita_chatbot:6 - Returning the final answer


The Bhagavad Gita describes the nature of the soul as the most important part of the body, giving it value and significance. According to the Gita, the body is perishable, while the soul is not. The soul is distinct from the material body and is characterized by eternal life, bliss, and knowledge, which are transcendental and beyond both body and mind. The Gita emphasizes the importance of understanding the soul's constitutional position and its relationship with the Supersoul, which can be realized through the path of knowledge, the practice of the eightfold system, or by bhakti-yoga.


In [ ]:
answer_5 = doctor_gpt_ai(user_query=question_5)
print(answer_5)

2025-05-23 07:06:10.693 | INFO     | __main__:gita_chatbot:2 - Searching for similar docs in the Bhagavad Gita vector DB
2025-05-23 07:06:11.058 | INFO     | __main__:gita_chatbot:4 - Calling the gitabot_ai to get answer
2025-05-23 07:06:11.843 | INFO     | __main__:gita_chatbot:6 - Returning the final answer


The significance of devotion (bhakti) according to the Gita is that it is a deliberate means of religious satisfaction, not a mere outpouring of poetic emotion. Devotion is considered a specific benediction of devotional service, and simply by chanting the holy name of Krsna, a devotee of the Lord can approach the supreme destination easily and happily. This destination cannot be approached by any other process of religion. Bhakti-yoga or transcendental loving service begins with determined faith, called drdha-vrata, which is the verdict of all scriptures.


In [ ]:
test_5 = doctor_gpt_ai("")
print(test_5)

2025-05-23 07:06:47.020 | INFO     | __main__:gita_chatbot:2 - Searching for similar docs in the Bhagavad Gita vector DB
2025-05-23 07:06:47.364 | INFO     | __main__:gita_chatbot:4 - Calling the gitabot_ai to get answer
2025-05-23 07:06:47.679 | INFO     | __main__:gita_chatbot:6 - Returning the final answer


Please provide a valid query.
